In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../')

from pprint import pprint
import numpy as np
import descarteslabs as dl

from src.ReMasFrame import *

In [ ]:
nasa_df = ReMasFrame()

In [6]:
# Choose an idx --> a landslide (or filter the geodataframe :D)
idx_test = 11031

# Returns products that we use as a dict
products = nasa_df.get_products()

nasa_df.loc[idx_test, :]

In [ ]:
pprint(products['weather']['cfs'])

In [ ]:
def get_scenes(product, buffer_size, res):
    # Returns start and end date of a 4 day interval
    start_date, end_date = nasa_df.date_interval(nasa_df.event_date[idx_test], delta_minus=10)
    
    scenes, ctx = ReMasFrame.search_scenes(
        nasa_df['geometry'][idx_test].buffer(buffer_size).envelope, 
        product['id'], 
        start_date=start_date, 
        end_date=end_date, 
        limit=20
    )
    
    new_ctx = ctx.assign(resolution=res)
    
    return scenes, new_ctx, start_date, end_date

def get_composite(product, buffer_size, res):
    
    scenes, new_ctx = get_scenes(product, buffer_size, res)
    
    arr_stack = scenes.stack(product['bands'], new_ctx)
    composite = np.ma.median(arr_stack, axis=0)
    
    return (scenes, new_ctx), composite

In [ ]:
products['weather']['cfs']['bands'], len(products['weather']['cfs']['bands'])

In [ ]:
idx_soil_bands = []
for idx, band in enumerate(products['weather']['cfs']['bands']):
    if 'soil' in band:
        idx_soil_bands.append(idx)
idx_soil_bands

In [ ]:
scenes, ctx, ini, end = get_scenes(
    products['weather']['cfs'], 
    buffer_size=0.1, #  10kmx10km
    res=0.20         # 0.2 deg x pix approx 20km x pix which is resolution of the sensor
)
scene_stack = scenes.stack(products['weather']['cfs']['bands'], ctx)
scene_stack.shape

Create new soil_moist band

In [ ]:
soil_bands = scene_stack[:, idx_soil_bands, ...]

In [12]:
old_shape = list(soil_bands.shape)
old_shape[1] = 1
new_shape = tuple(old_shape)
new_soil_moist = np.sum(soil_bands, axis=1).reshape(new_shape)

scene_stack = np.concatenate((scene_stack, new_soil_moist), axis=1)

new_product = np.delete(scene_stack, idx_soil_bands, axis=1)
new_product.shape

(11, 10, 2, 2)

**Shape is (batch, channels, x, y)**

In [1]:
from src.uploader import Uploader

upload = Uploader('dataset')
upload.fill_value(new_product)

ModuleNotFoundError: No module named 'src'